In [0]:
!pip install --upgrade tensorflow-gpu==1.14
!pip install --upgrade fire

     |████████████████████████████████| 377.0MB 143kB/s 
     |████████████████████████████████| 3.2MB 29.9MB/s 
     |████████████████████████████████| 491kB 38.2MB/s 
ERROR: tensorflow 1.15.0rc3 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.14.0 which is incompatible.
ERROR: tensorflow 1.15.0rc3 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.14.0 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
     |████████████████████████████████| 81kB 3.1MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=055a0dea2095aceb46d788118c5d3cf5178a1bb53598d8238e5142e4f8a5494a
  Stored in directory: /root/.cache/pip/wheels/31/9c/

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


# Opens the project folder. IMPORTANT: Change to your route
%cd 'drive/My Drive/TEC/2019_2S/IA/TareaProgramada2'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/TEC/2019_2S/IA/TareaProgramada2


In [0]:
import tensorflow as tf
import fire
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [0]:
print ('x_train shape:', x_train.shape)
print ('y_train shape: ', y_train.shape)
print ('x_test shape: ', x_test.shape)
print ('y_test shape:', y_test.shape)

print('muestras de entrenamiento: ', x_train.shape[0])
print('muestras de test: ', x_test.shape[0])


x_train shape: (50000, 32, 32, 3)
y_train shape:  (50000, 1)
x_test shape:  (10000, 32, 32, 3)
y_test shape: (10000, 1)
muestras de entrenamiento:  50000
muestras de test:  10000


In [0]:
class Train:
  _x_ = []
  _y_ = []
  _logits = []
  _loss = []
  _train_step = []
  _merged_summary_op = []
  _saver = []
  _session = []
  _writer = []
  _is_training = []
  _loss_val = []
  _train_summary = []
  _val_summary = []
  
  def _init_(self):
    pass
  

  def build_graph(self):
    self._x_ = tf.placeholder("float", shape=[None, 32, 32, 3], name='X')
    self._y_ = tf.placeholder("int32", shape=[None, 10], name='Y')
    self._is_training = tf.placeholder(tf.bool)

    with tf.name_scope("model") as scope:

      #arquitecuta de la RED GoogleNet Inception v1
      
      conv1_7x7_S2 = tf.layers.conv2d(inputs=self._x_,filters=64, kernel_size=[7,7], strides=2,  padding="same", activation=tf.nn.relu)
      maxPool_3x3_S2 = tf.layers.max_pooling2d(inputs=conv1_7x7_S2, pool_size=[3,3], strides=2)
      norm1 = tf.nn.lrn(input=maxPool_3x3_S2, bias=1.0, alpha=0.001 / 9.0, beta=0.75,name='norm1' )
      conv_1x1_V1 = tf.layers.conv2d(norm1,filters=64, kernel_size=[1,1], strides=1, padding="same", activation=tf.nn.relu)
      conv_3x3_S1 = tf.layers.conv2d(conv_1x1_V1,filters=192, kernel_size=[3,3], strides=1, padding="same", activation=tf.nn.relu)
      norm2 = tf.nn.lrn(input=conv_3x3_S1,  bias=1.0, alpha=0.001 / 9.0, beta=0.75,name='norm2' )
      
      maxPool2_3x3_S2 = tf.layers.max_pooling2d(inputs=norm2, pool_size=[3,3], strides=2)
      
      ###############primera capa de inception 3a
      conv3a_1x1redu3x3_S = tf.layers.conv2d(inputs=maxPool2_3x3_S2, filters=96, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv3a_1x1redu5x5_S = tf.layers.conv2d(inputs=maxPool2_3x3_S2, filters=16, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      maxPool3a_3x3_S = tf.layers.max_pooling2d(inputs=maxPool2_3x3_S2, pool_size=[3,3], strides=1)
      
      conv3a_1x1_S = tf.layers.conv2d(inputs=maxPool2_3x3_S2, filters=64, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv3a_3x3_S = tf.layers.conv2d(inputs=conv3a_1x1redu3x3_S, filters=128, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv3a_5x5_S = tf.layers.conv2d(inputs=conv3a_1x1redu5x5_S, filters=32, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv3aM_1x1_S = tf.layers.conv2d(inputs=maxPool3a_3x3_S, filters=32, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      #################DEPTH CONCAT
      
      #intento de concat 
      print("todo bien hasta aquí 1")
      #print(conv3a_1x1_S.shape,conv3a_1x1_S.shape,conv3a_3x3_S.shape,conv3a_5x5_S.shape,conv3aM_1x1_S.shape)

      concat1 = tf.concat([conv3a_1x1_S,conv3a_3x3_S,conv3a_5x5_S,conv3aM_1x1_S],axis=-1,name='concat1')
      #concat1 = tf.concat([conv3a_1x1_S,conv3a_3x3_S,conv3a_5x5_S],axis=-1)
      print("todo bien hasta aquí 1.1")
      print(concat1.shape)
      #############################Inception 3b
      conv3b_1x1redu3x3_S = tf.layers.conv2d(inputs=concat1, filters=128, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv3b_1x1redu5x5_S = tf.layers.conv2d(inputs=concat1, filters=32, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      maxPool3b_3x3_S = tf.layers.max_pooling2d(inputs=concat1, pool_size=[3,3], strides=1)
      
      conv3b_1x1_S = tf.layers.conv2d(inputs=maxPool2_3x3_S2, filters=128, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv3b_3x3_S = tf.layers.conv2d(inputs=conv3a_1x1redu3x3_S, filters=192, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv3b_5x5_S = tf.layers.conv2d(inputs=conv3a_1x1redu5x5_S, filters=96, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv3bM_1x1_S = tf.layers.conv2d(inputs=maxPool3a_3x3_S, filters=64, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      #################DEPTH CONCAT
      print("todo bien hasta aquí 2")
      #hacel el concat 2
      
      #intento de concat 

      concat2 = tf.concat([conv3b_1x1_S, conv3b_3x3_S, conv3b_5x5_S, conv3bM_1x1_S],axis=1)



      ############Maxpool 3
      maxPool3_3x3_2S = tf.layers.max_pooling2d(inputs=concat2, pool_size=[3,3], strides=2)
      
      
      
      ####Inception 4a###########################
      conv4a_1x1redu3x3_S = tf.layers.conv2d(inputs=maxPool3_3x3_2S, filters=96, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv4a_1x1redu5x5_S = tf.layers.conv2d(inputs=maxPool3_3x3_2S, filters=16, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      maxPool4a_3x3_S = tf.layers.max_pooling2d(inputs=maxPool3_3x3_2S, pool_size=[3,3], strides=1)
      
      conv4a_1x1_S = tf.layers.conv2d(inputs=maxPool3_3x3_2S, filters=192, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv4a_3x3_S = tf.layers.conv2d(inputs=conv4a_1x1redu3x3_S, filters=208, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv4a_5x5_S = tf.layers.conv2d(inputs=conv4a_1x1redu5x5_S, filters=48, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv4aM_1x1_S = tf.layers.conv2d(inputs=maxPool4a_3x3_S, filters=64, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      
      
      print("todo bien hasta aquí 3")
      #hacer el concat 3
      #intento de concat 

      concat3 = tf.concat([conv4a_1x1_S, conv4a_3x3_S, conv4a_5x5_S,conv4aM_1x1_S],axis=1)
     
      
      
      
      #############################Inception 4b
      conv4b_1x1redu3x3_S = tf.layers.conv2d(inputs=concat3, filters=112, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv4b_1x1redu5x5_S = tf.layers.conv2d(inputs=concat3, filters=24, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      maxPool4b_3x3_S = tf.layers.max_pooling2d(inputs=concat3, pool_size=[3,3], strides=1)
      averagePool4b_5x5_3V = tf.layers.average_pooling2d(inputs=concat3, pool_size=[5,5], strides=3)
      
      conv4b_1x1_S = tf.layers.conv2d(inputs=concat3, filters=160, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv4b_3x3_S = tf.layers.conv2d(inputs=conv4b_1x1redu3x3_S, filters=224, kernel_size=[3,3],strides=1, padding="same", activation=tf.nn.relu)
      conv4b_5x5_S = tf.layers.conv2d(inputs=conv4b_1x1redu5x5_S, filters=64, kernel_size=[5,5],strides=1, padding="same", activation=tf.nn.relu)
      conv4bM_1x1_S = tf.layers.conv2d(inputs=maxPool4b_3x3_S, filters=64, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      
      conv4bAP_1x1_S = tf.layers.conv2d(inputs=averagePool4b_5x5_3V, filters=160, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      
      FC_4b1 = tf.layers.dense(inputs=conv4bAP_1x1_S, units=1024, activation=tf.nn.relu)
      FC_4b2 = tf.layers.dense(inputs=FC_4b1, units=1000, activation=tf.nn.relu)
      softMax1= tf.keras.activations.softmax(FC_4b2 , axis=1)
      # aqui el concat 4

      #intento de concat 

      concat4 = tf.concat([conv4b_1x1_S, conv4b_3x3_S,conv4b_5x5_S ,conv4bM_1x1_S],axis=1)
      print("todo bien hasta aquí 4")
     
      

      ######inception 4c
      conv4c_1x1redu3x3_S = tf.layers.conv2d(inputs=concat4, filters=128, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv4c_1x1redu5x5_S = tf.layers.conv2d(inputs=concat4, filters=24, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      maxPool4c_3x3_S = tf.layers.max_pooling2d(inputs=concat4, pool_size=[3,3], strides=1)
      
      conv4c_1x1_S = tf.layers.conv2d(inputs=concat4, filters=128, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv4c_3x3_S = tf.layers.conv2d(inputs=conv4c_1x1redu3x3_S, filters=256, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv4c_5x5_S = tf.layers.conv2d(inputs=conv4c_1x1redu5x5_S, filters=64, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv4cM_1x1_S = tf.layers.conv2d(inputs=maxPool4c_3x3_S, filters=64, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      
      #####Concat5  intento
      concat5 = tf.concat([conv4c_1x1_S, conv4c_3x3_S,conv4c_5x5_S ,conv4cM_1x1_S],axis=1)
      
      #####inception 4d###
      conv4d_1x1redu3x3_S = tf.layers.conv2d(inputs=concat5, filters=144, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv4d_1x1redu5x5_S = tf.layers.conv2d(inputs=concat5, filters=32, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      maxPool4d_3x3_S = tf.layers.max_pooling2d(inputs=concat5, pool_size=[3,3], strides=1)
      
      conv4d_1x1_S = tf.layers.conv2d(inputs=concat5, filters=112, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv4d_3x3_S = tf.layers.conv2d(inputs=conv4d_1x1redu3x3_S, filters=288, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv4d_5x5_S = tf.layers.conv2d(inputs=conv4d_1x1redu5x5_S, filters=64, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv4dM_1x1_S = tf.layers.conv2d(inputs=maxPool4d_3x3_S, filters=64, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      
      ###Concat6 intento
      
      concat6 = tf.concat([conv4d_1x1_S, conv4d_3x3_S,conv4d_5x5_S ,conv4dM_1x1_S],axis=1)
      
      ##########inception 4e#####
      conv4e_1x1redu3x3_S = tf.layers.conv2d(inputs=concat6, filters=160, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv4e_1x1redu5x5_S = tf.layers.conv2d(inputs=concat6, filters=32, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      maxPool4e_3x3_S = tf.layers.max_pooling2d(inputs=concat6, pool_size=[3,3], strides=1)
      averagePool4e_5x5_3V = tf.layers.average_pooling2d(inputs=concat3, pool_size=[5,5], strides=3)
      
      conv4e_1x1_S = tf.layers.conv2d(inputs=concat6, filters=256, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv4e_3x3_S = tf.layers.conv2d(inputs=conv4e_1x1redu3x3_S, filters=320, kernel_size=[3,3],strides=1, padding="same", activation=tf.nn.relu)
      conv4e_5x5_S = tf.layers.conv2d(inputs=conv4e_1x1redu5x5_S, filters=128, kernel_size=[5,5],strides=1, padding="same", activation=tf.nn.relu)
      conv4eM_1x1_S = tf.layers.conv2d(inputs=maxPool4e_3x3_S, filters=128, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      
      conv4eAP_1x1_S = tf.layers.conv2d(inputs=averagePool4e_5x5_3V, filters=160, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      
      FC_4e1 = tf.layers.dense(inputs=conv4eAP_1x1_S, units=1024, activation=tf.nn.relu)
      FC_4e2 = tf.layers.dense(inputs=FC_4e1, units=1000, activation=tf.nn.relu)
      
      
      #################concat7#############################
      print("todo bien hasta aquí 5")
      concat7 = tf.concat([conv4e_1x1_S, conv4e_3x3_S,conv4e_5x5_S ,conv4eM_1x1_S],axis=1)
      
      ####################Maxpool4
      maxPool4_3x3_2S = tf.layers.max_pooling2d(inputs=concat7, pool_size=[3,3], strides=2)
      
      #############inception5a#############
      conv5a_1x1redu3x3_S = tf.layers.conv2d(inputs=maxPool4_3x3_2S, filters=160, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv5a_1x1redu5x5_S = tf.layers.conv2d(inputs=maxPool4_3x3_2S, filters=32, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      maxPool5a_3x3_S = tf.layers.max_pooling2d(inputs=maxPool4_3x3_2S, pool_size=[3,3], strides=1)
      
      conv5a_1x1_S = tf.layers.conv2d(inputs=maxPool4_3x3_2S, filters=256, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv5a_3x3_S = tf.layers.conv2d(inputs=conv5a_1x1redu3x3_S, filters=320, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv5a_5x5_S = tf.layers.conv2d(inputs=conv5a_1x1redu5x5_S, filters=128, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv5aM_1x1_S = tf.layers.conv2d(inputs=maxPool5a_3x3_S, filters=128, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      
      
      ###############concat8######################## intento
      
      concat8 = tf.concat([conv5a_1x1_S, conv5a_3x3_S,conv5a_5x5_S ,conv5aM_1x1_S],axis=1)
      
      
      ###################inception5b
      
      conv5b_1x1redu3x3_S = tf.layers.conv2d(inputs=concat8, filters=192, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv5b_1x1redu5x5_S = tf.layers.conv2d(inputs=concat8, filters=48, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      maxPool5b_3x3_S = tf.layers.max_pooling2d(inputs=concat8, pool_size=[3,3], strides=1)
      
      conv5b_1x1_S = tf.layers.conv2d(inputs=concat8, filters=384, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv5b_3x3_S = tf.layers.conv2d(inputs=conv5b_1x1redu3x3_S, filters=384, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv5b_5x5_S = tf.layers.conv2d(inputs=conv5b_1x1redu5x5_S, filters=128, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      conv5bM_1x1_S = tf.layers.conv2d(inputs=maxPool5b_3x3_S, filters=128, kernel_size=[1,1],strides=1, padding="same", activation=tf.nn.relu)
      
      ####################concat9##############
      print("todo bien hasta aquí 6 ")
      
      concat9 = tf.concat([conv5b_1x1_S, conv5b_3x3_S,conv5b_5x5_S ,conv5bM_1x1_S],axis=1)
      
      #########averagepoolF1
      
      averagePoolF1_7x7_V = tf.layers.average_pooling2d(inputs=concat9, pool_size=[7,7], strides=1)
      
      #############FCF
      
      FCF= tf.layers.dense(inputs=averagePoolF1_7x7_V, units=1024, activation=tf.nn.relu)
      
 
       
      ##########Softmax final
      softMax2= tf.keras.activations.softmax(FCF, axis=1)
      
      
      
      
      
      ##############################################################################################################

      self._logits = tf.layers.dense(inputs=softMax2, units=10)

#flaggggggggggggggggggggggggggggggggggggggggggggggggggg

      print("todo bien hasta aquí x2")

#flaggggggggggggggggggggggggggggggggggggggggggggggggggg

      #hola
      
      with tf.name_scope("loss_func") as scope:

        self._loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self._logits, labels=self._y_))
        self._loss_val = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self._logits, labels=self._y_))

        self._train_summary =tf.summary.scalar("loss_train",self._loss)
        self._val_summary =tf.summary.scalar("loss_val",self._loss_val)

      
      
      with tf.name_scope("optimizer") as scope:

        global_step = tf.Variable(0, trainable=False)
        start_learning_rate = 1e-3
        learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 1000, 0.9, staircase=True)

        self._train_step = tf.train.AdamOptimizer(learning_rate).minimize(self._loss, global_step=global_step)

        tf.summary.scalar("learning rate", learning_rate)
        tf.summary.scalar("global step", global_step)

      self._merged_summary_op = tf.summary.merge_all()
      init = tf.global_variables_initializer()
      self._saver = tf.train.Saver(max_to_keep=None)
      
      
      
    

      gpu_options = tf.GPUOptions(allow_growth = True)

      self._session = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

      self._writer = tf.summary.FileWriter("./logs/cifar10", self._session.graph)

      self._session.run(init)
    
 

  def train(self, save_dir=os.getcwd(), batch_size=750):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)

          

    dataset_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    dataset_train = dataset_train.shuffle(buffer_size=10000)
    dataset_train = dataset_train.repeat()
    dataset_train = dataset_train.batch(batch_size)
    dataset_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    dataset_test = dataset_test.repeat()
    dataset_test = dataset_test.batch(batch_size)

          

    iter_train = dataset_train.make_one_shot_iterator()
    iter_train_op = iter_train.get_next()
    iter_test = dataset_test.make_one_shot_iterator()
    iter_test_op = iter_test.get_next()

    self.build_graph()



    for i in range(200):

      batch_train = self._session.run([iter_train_op])
      batch_x_train, batch_y_train = batch_train[0]

      if i % 10 == 0:
        batch_test = self._session.run([iter_test_op])
        batch_x_test, batch_y_test = batch_test[0]

        loss_train, summary_1 = self._session.run([self._loss, self._merged_summary_op], feed_dict={self._x_:batch_x_train, self._y_:batch_y_train, self._is_training:True})

        loss_val, summary_2 = self._session.run([self._loss_val, self._val_summary], feed_dict={self._x_:batch_x_test, self._y_:batch_y_test, self._is_training:False})

        print("Epoch: {0}, Loss Train: {1} Loss Val: {2}". format(i, loss_train, loss_val))

        self._writer.add_summary(summary_1, i)
        self._writer.add_summary(summary_2, i)

      self._train_step.run(session=self._session, feed_dict={self._x_:batch_x_train, self._y_:batch_y_train, self._is_training:True})

      if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
      checkpoint_path = os.path.join(save_dir, "model")
      filename = self._saver.save(self._session, checkpoint_path)


if __name__ == '__main__':
  cnn=Train()
  cnn.train

cnn.train()


todo bien hasta aquí 1


ValueError: ignored

In [0]:
cnn._saver.save(cnn._session, 'my-model-test')

'my-model-test'